# Results Tables

The success of an assembly is evaluated by the computation of metrics in two defined ways: globally through statistics inherent to the complete set of sequences that were assembled, and relative to the replicons present in the sample. 

The computation of the global metrics is done through custom python code from the complete set of sequences assembled. 

The following metrics are computed for the complete and filtered set of assembled sequences, restricted to contigs of length above a specified minimum size: 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Maximum sequence length:** The length of the largest contig in the assembly.
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **Contiguity**
    - **Nx (where 0  < x  ⩽ 100):** Length for which the collection of all assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the assembly
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event

After filtering, the sequences are mapped with the reference replicons and the metrics are computed through custom python code. The metrics are calculated, filtered for a minimum length, for each replicon in the file provided input references. 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **COMPASS**
    - **(Breadth of) Coverage:** Ratio of covered sequence on the reference by aligned contigs;
    - **Multiplicity:** Ratio of the length of alignable assembled sequence to covered sequence on the reference;
    - **Validity:** Ratio of the length of the alignable assembled sequence to total basepairs in the aligned contigs;
    - **Parsimony:** Cost of the assembly (multiplicity over validity);
- **Contiguity**
    - **Contiguity:** longest single alignment between the assembly and the reference, relative to the reference length;
    - **NAx (where 0  < x  ⩽ 100):** Length for which the collection of aligned assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the reference replicon;
    - **NGx (where 0  < x  ⩽ 100):** Length for which the collection of aligned contigs of that length or longer covers at least a given percentage of the sequence of the reference.
    - **Lx (where 0  < x  ⩽ 100):** Minimal number of contigs that cover x % of the sequence of the reference;
- **Identity**
    - **Identity:** Ratio of identical basepairs in all aligned contigs to the reference;
    - **Lowest identity:** Identity of the lowest scoring contig to the reference.
    - **PLS**: Phred-like score per contig, per assembler.
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event


## Imports

In [1]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np

## Global variables

In [2]:
METRICS_COLUMNS_GLOBAL = ['run','sample','assembler', 'contigs', 'basepairs', 'max_contig', 'Ns', 'n50', 
                          'misassemblies', 'mapped_reads', 'filtered_contigs', 'filtered_basepairs',
                          'filtered_n50', 'filtered_Ns', 'type', 'algorythm']
METRICS_COLUMNS_REFERENCE = ['run','sample','assembler', 'reference', 'contiguity', 'breadth_of_coverage', 'multiplicity', 'validity', 
                             'parsimony', 'identity', 'lowest_identity', 'L90', 'contigs', 'NA50', 'NG50', 'basepairs', 'Ns',
                             'misassemblies', 'type', 'algorythm']

ASSEMBLER_PROCESS_LIST = ["BCALM2", "GATBMINIAPIPELINE", "MINIA", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "VELVETOPTIMIZER", "IDBA"]
PROCESS_TO_NAME = {"BCALM2": "BCALM2", 
                   "GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "MINIA": "MINIA",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                    "SKESA": "SKESA",
                   "VELVETOPTIMIZER": "VelvetOptimizer",
                   "IDBA": "IDBA-UD"}

log_distributed = ['LHS', 'LNN', 'ERR2935805']
even_distribution = ['EHS', 'LNN', 'ERR2984773']

genomic_assemblers = ['BCALM2', 'MINIA', 'SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer']
metagenomic_assemblers = ['GATBMiniaPipeline', 'IDBA-UD', 'MEGAHIT', 'metaSPAdes']
single_kmer = ['BCALM2', 'MINIA']
multiple_kmer = ['SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer', 'GATBMiniaPipeline', 
                 'IDBA-UD', 'MEGAHIT', 'metaSPAdes']

## Global metrics

### Load data

In [3]:
report_glob = glob.glob('../Results/*/report/pipeline_report_tables.json')
global_pipeline_metrics_df = pd.DataFrame(columns=METRICS_COLUMNS_GLOBAL)

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for line in json_report[sample]['GlobalTable']:
                assembler = line['assembler']
                global_pipeline_metrics_df = global_pipeline_metrics_df.append({'run': stats_run,
                                                                                'sample': sample,
                                                                                'assembler': line['assembler'],
                                                                                'contigs': int(line['original']['contigs']),
                                                                                'basepairs': int(line['original']['basepairs']),
                                                                                'max_contig': int(line['original']['max_contig_size']),
                                                                                'n50': int(line['original']['N50']),
                                                                                'mapped_reads': line['original']['mapped_reads'],
                                                                                'Ns': int(line['original']['Ns']),
                                                                                'misassemblies': line['filtered']['misassembled_contigs'],
                                                                                'filtered_contigs': line['filtered']['contigs'],
                                                                                'filtered_basepairs': line['filtered']['basepairs'],
                                                                                'filtered_n50': line['filtered']['N50'],
                                                                                'filtered_Ns': line['filtered']['Ns']
                                                                               },
                                                                               ignore_index=True)
global_pipeline_metrics_df['distribution'] = np.where(global_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
global_pipeline_metrics_df['type'] = np.where(global_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
global_pipeline_metrics_df['algorythm'] = np.where(global_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50','misassemblies']] = global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50','misassemblies']].apply(pd.to_numeric)

Processing pipeline_report_tables.json data from run1...
Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...


### Preview data

In [4]:
global_pipeline_metrics_df.dtypes

run                    object
sample                 object
assembler              object
contigs                 int64
basepairs               int64
max_contig              int64
Ns                      int64
n50                     int64
misassemblies           int64
mapped_reads          float64
filtered_contigs       object
filtered_basepairs     object
filtered_n50           object
filtered_Ns            object
type                   object
algorythm              object
distribution           object
dtype: object

In [5]:
global_pipeline_metrics_df

,run,sample,assembler,contigs,basepairs,max_contig,Ns,n50,misassemblies,mapped_reads,filtered_contigs,filtered_basepairs,filtered_n50,filtered_Ns,type,algorythm,distribution
0,run1,ENN,BCALM2,25258,31040696,161812,0,17766,0,96.832853,3542,28188554,21029,0,Genomic,Single k-mer De Bruijn graph,Even
1,run1,ENN,GATBMiniaPipeline,1008,30445418,632215,0,106963,0,99.480322,727,30282639,106963,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
2,run1,ENN,IDBA-UD,1825,30390287,468605,0,62331,13,99.414101,1279,30136999,63253,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
3,run1,ENN,MEGAHIT,810,30541296,632274,0,170162,9,99.731491,517,30415305,171359,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
4,run1,ENN,metaSPAdes,633,30448137,1036942,0,234919,1,99.596146,350,30376419,234919,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,run2,LHS,MINIA,24662,15334974,163543,0,7727,0,91.098929,2172,12922094,10582,0,Genomic,Single k-mer De Bruijn graph,Log
176,run2,LHS,SKESA,36,2957060,632013,0,351027,0,92.228441,21,2950393,351027,0,Genomic,Multiple k-mer De Bruijn graph,Log
177,run2,LHS,SPAdes,31225,16224766,263645,0,46478,1,95.503607,1526,13088051,69361,0,Genomic,Multiple k-mer De Bruijn graph,Log
178,run2,LHS,Unicycler,276,13345058,632062,0,187479,3,96.948570,247,13333597,187479,0,Genomic,Multiple k-mer De Bruijn graph,Log


### Global statistics per assembler type

In [6]:
stats_per_assembler_type = pd.DataFrame()
for assembly_type in global_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.type == assembly_type].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    stats_per_assembler_type=stats_per_assembler_type.append(row, ignore_index=True)
stats_per_assembler_type = stats_per_assembler_type.set_index('Assembler Type')
stats_per_assembler_type.to_csv("Tables/Results/Global metrics per assembler type.csv")

In [7]:
stats_per_assembler_type

,Ns,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler Type,,,,,,,
Genomic,632.23(±2690.54),45034003.1(±77233098.8),584983.59(±1922998.87),74.63(±36.34),407391.44(±369087.17),18.17(±94.33),75438.73(±96063.55)
Metagenomic,0.0(±0.0),25546894.07(±9976656.57),8085.12(±13481.19),94.46(±6.08),706300.71(±215974.24),4.06(±4.63),116904.35(±64562.91)


### Global stats per assembly algorithm

In [8]:
stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in global_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    stats_per_assembler_algo=stats_per_assembler_algo.append(row, ignore_index=True)
stats_per_assembler_algo = stats_per_assembler_algo.set_index('Assembler Algorithm')
stats_per_assembler_algo.to_csv("Tables/Results/Global metrics per assembler algorithm.csv")

In [9]:
stats_per_assembler_algo

,Ns,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler Algorithm,,,,,,,
Single k-mer De Bruijn graph,0.0(±0.0),91615770.47(±120234711.8),1691066.5(±3063933.44),68.91(±38.27),108171.83(±74130.16),0.33(±0.76),10205.89(±8991.9)
Multiple k-mer De Bruijn graph,474.17(±2343.51),23645006.74(±12596918.3),20013.63(±86808.16),85.97(±26.61),631650.97(±309107.29),15.57(±81.74),112479.75(±85835.01)


### Global stats per assembler

In [10]:
stats_per_assembler = pd.DataFrame()
for assembler in global_pipeline_metrics_df.assembler.unique():
    row={'Assembler': assembler, 
         'Type':global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].type.unique()[0],
         'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    stats_per_assembler=stats_per_assembler.append(row, ignore_index=True)
stats_per_assembler = stats_per_assembler.set_index('Assembler')
stats_per_assembler.to_csv("Tables/Results/Global metrics per assembler.csv")

In [11]:
stats_per_assembler

,Algorithm,Ns,Type,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,157403460.17(±143207783.36),3352423.83(±3671823.35),46.33(±43.54),58728.17(±75986.47),0.0(±0.0),6220.67(±8741.64)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,23073291.67(±8534830.43),2629.61(±2551.95),94.65(±6.43),640091.33(±67932.83),1.83(±2.09),102416.83(±26459.85)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,25378796.44(±9555093.25),6832.39(±7740.59),93.22(±6.12),515095.83(±119980.22),6.22(±4.73),68440.89(±30988.57)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,25840461.5(±9906281.89),6528.5(±8141.67),94.98(±6.1),847862.5(±225298.59),6.33(±6.39),147830.83(±47068.49)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,27895026.67(±11873670.3),16350.0(±22765.29),94.97(±6.0),822153.17(±215081.62),1.83(±0.92),148928.83(±93410.42)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,25828080.78(±9458287.51),29709.17(±19273.53),91.49(±6.34),157615.5(±19050.12),0.67(±0.97),14191.11(±7511.05)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,18472455.67(±12305308.04),3671.5(±4029.8),90.6(±8.05),390219.83(±260046.23),3.33(±4.39),101495.67(±121656.06)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,28138604.33(±10712564.41),19436.5(±17118.19),94.21(±6.4),697457.83(±215190.75),98.0(±218.61),107840.17(±84233.07)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,21030768.0(±9764955.84),531.33(±325.41),94.07(±6.74),857592.0(±230735.33),1.33(±1.41),212762.33(±21896.83)


### Global stats per distribution

In [25]:
for distribution in reference_pipeline_metrics_df.distribution.unique():
    stats_per_distribution = pd.DataFrame()
    for assembler in global_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].type.unique()[0],
             'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            std = lala.loc['std',column]
            row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
        stats_per_distribution=stats_per_distribution.append(row, ignore_index=True)
    stats_per_distribution = stats_per_distribution.set_index('Assembler')
    stats_per_distribution.to_csv("Tables/Results/{} - Global metrics per assembler.csv".format(distribution))
    print(distribution)
    display(stats_per_distribution)

Even


,Algorithm,Ns,Type,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,81485532.0(±66888830.04),1286040.67(±1671523.11),57.29(±39.07),61507.0(±75761.93),0.0(±0.0),6186.0(±8690.2)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,31254751.44(±1167859.11),2439.78(±2220.06),93.87(±8.05),677707.0(±68066.32),2.67(±2.65),110537.33(±9239.15)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,33523214.0(±4692045.42),7943.22(±9186.33),90.89(±6.89),413513.0(±82638.0),9.78(±4.35),49109.78(±21114.03)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,34099350.67(±5300334.81),7463.33(±9863.23),94.35(±7.6),868834.0(±259534.83),12.0(±3.77),143276.33(±41229.51)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,36624891.67(±9252202.76),21002.33(±30476.03),94.28(±7.47),847661.67(±280794.32),2.0(±0.87),158678.0(±88651.34)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,34158216.89(±5270643.61),33769.89(±25663.37),90.39(±7.57),147189.67(±21933.5),0.0(±0.0),14482.0(±5545.5)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,30285330.33(±112007.2),1219.0(±192.79),92.53(±8.46),540293.0(±80495.71),2.67(±2.0),85120.33(±16388.81)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,35828349.33(±8031371.09),13949.33(±19394.94),94.27(±7.53),813593.33(±51167.5),4.33(±4.27),158629.67(±80482.08)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,30462753.0(±96627.2),833.67(±134.55),93.41(±8.33),1011262.0(±219958.36),1.67(±1.32),215168.67(±26065.96)


Log


,Algorithm,Ns,Type,basepairs,contigs,mapped_reads,max_contig,misassemblies,n50
Assembler,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,233321388.33(±161673853.51),5418807.0(±4030943.57),35.37(±47.25),55949.33(±80699.36),0.0(±0.0),6255.33(±9320.0)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,14891831.89(±1679744.93),2819.44(±2971.41),95.43(±4.65),602475.67(±44609.0),1.0(±0.87),94296.33(±35412.28)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,17234378.89(±4769784.15),5721.56(±6337.04),95.56(±4.46),616678.67(±23301.5),2.67(±0.5),87772.0(±27459.24)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,17581572.33(±5193388.33),5593.67(±6450.57),95.61(±4.51),826891.0(±198786.22),0.67(±0.5),152385.33(±54417.57)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Metagenomic,19165161.67(±6520285.33),11697.67(±11127.06),95.66(±4.44),796644.67(±134138.83),1.67(±1.0),139179.67(±102317.1)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),Genomic,17497944.67(±2488170.91),25648.44(±9677.9),92.59(±5.04),168041.33(±6747.5),1.33(±1.0),13900.22(±9430.85)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,6659581.0(±2789663.83),6124.0(±4575.82),88.66(±7.6),240146.67(±294188.94),4.0(±6.0),117871.0(±174867.04)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,20448859.33(±6805338.32),24923.67(±13371.06),94.15(±5.5),581322.33(±255807.76),191.67(±286.0),57050.67(±52876.44)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),Genomic,11598783.0(±1566338.53),229.0(±35.38),94.74(±5.12),703922.0(±107787.75),1.0(±1.5),210356.0(±18066.91)


## Global metrics

### Load data

In [12]:
report_glob = glob.glob('../Results/*/report/pipeline_report_tables.json')
reference_pipeline_metrics_df = pd.DataFrame(columns=METRICS_COLUMNS_REFERENCE)

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for reference, data in json_report[sample]['ReferenceTables'].items():
                for row in data:
                    for item in row:
                        reference_pipeline_metrics_df = reference_pipeline_metrics_df.append({'run': stats_run,
                                                                                        'sample': sample,
                                                                                        'assembler': item['assembler'],
                                                                                        'reference': reference,
                                                                                        'contiguity': item['contiguity'],
                                                                                        'breadth_of_coverage': item['breadth_of_coverage'],
                                                                                        'multiplicity': item['multiplicity'],
                                                                                        'validity': item['validity'],
                                                                                        'parsimony': item['parsimony'],
                                                                                        'identity': item['identity'],
                                                                                        'lowest_identity': item['lowest_identity'],
                                                                                        'L90': item['L90'],
                                                                                        'contigs': item['aligned_contigs'],
                                                                                        'NA50': item['NA50'],
                                                                                        'NG50': item['NG50'],
                                                                                        'basepairs': item['aligned_basepairs'],
                                                                                        'Ns': item['Ns'],
                                                                                        'misassemblies': item['misassembled_contigs']},
                                                                                       ignore_index=True)

reference_pipeline_metrics_df['distribution'] = np.where(reference_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
reference_pipeline_metrics_df['type'] = np.where(reference_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
reference_pipeline_metrics_df['algorythm'] = np.where(reference_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassemblies']] = reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassemblies']].apply(pd.to_numeric)
display(reference_pipeline_metrics_df)

Processing pipeline_report_tables.json data from run1...
Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...


,run,sample,assembler,reference,contiguity,breadth_of_coverage,multiplicity,validity,parsimony,identity,...,L90,contigs,NA50,NG50,basepairs,Ns,misassemblies,type,algorythm,distribution
0,run1,ENN,BCALM2,Bacillus_subtilis,0.032648,0.976835,1.000279,1.000000,1.000279,1.000000,...,66,116,65138,64934,3951958,0,0,Genomic,Single k-mer De Bruijn graph,Even
1,run1,ENN,GATBMiniaPipeline,Bacillus_subtilis,0.097833,0.983819,1.000000,1.000000,1.000000,0.999966,...,14,29,269695,263366,3980213,0,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
2,run1,ENN,IDBA-UD,Bacillus_subtilis,0.077894,0.982940,1.000048,0.999999,1.000048,0.999558,...,21,38,211169,211169,3976658,0,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
3,run1,ENN,MEGAHIT,Bacillus_subtilis,0.097848,0.985245,1.000233,0.999861,1.000372,0.998712,...,14,28,263587,263587,3985982,0,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
4,run1,ENN,metaSPAdes,Bacillus_subtilis,0.255293,0.985404,1.000000,1.000000,1.000000,0.998464,...,10,20,325068,325068,3986625,0,0,Metagenomic,Multiple k-mer De Bruijn graph,Even
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,run2,LHS,MINIA,Staphylococcus_aureus_plasmid3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,Genomic,Single k-mer De Bruijn graph,Log
2156,run2,LHS,SKESA,Staphylococcus_aureus_plasmid3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,Genomic,Multiple k-mer De Bruijn graph,Log
2157,run2,LHS,SPAdes,Staphylococcus_aureus_plasmid3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,Genomic,Multiple k-mer De Bruijn graph,Log
2158,run2,LHS,Unicycler,Staphylococcus_aureus_plasmid3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,Genomic,Multiple k-mer De Bruijn graph,Log


In [13]:
reference_pipeline_metrics_df.dtypes

run                     object
sample                  object
assembler               object
reference               object
contiguity             float64
breadth_of_coverage    float64
multiplicity           float64
validity               float64
parsimony              float64
identity               float64
lowest_identity        float64
L90                      int64
contigs                  int64
NA50                     int64
NG50                     int64
basepairs                int64
Ns                       int64
misassemblies            int64
type                    object
algorythm               object
distribution            object
dtype: object

### Reference statistics per assembler type

In [14]:
ref_stats_per_assembler_type = pd.DataFrame()
for assembly_type in reference_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.type == assembly_type].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    ref_stats_per_assembler_type=ref_stats_per_assembler_type.append(row, ignore_index=True)
ref_stats_per_assembler_type = ref_stats_per_assembler_type.set_index('Assembler Type')
ref_stats_per_assembler_type.to_csv("Tables/Results/Reference metrics per assembler type.csv")

In [15]:
ref_stats_per_assembler_type

,L90,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler Type,,,,,,,,,,,,,,
Genomic,32.26(±116.1),44530.55(±91863.12),44301.7(±93231.26),81.46(±800.9),1392592.69(±2023387.45),0.42(±0.47),113.05(±296.65),0.08(±0.2),0.5(±0.5),0.47(±0.49),1.55(±27.22),0.5(±0.5),0.51(±0.51),0.5(±0.5)
Metagenomic,24.21(±51.75),82729.11(±123470.96),81991.75(±122491.49),0.0(±0.0),1830607.91(±2235695.27),0.55(±0.49),45.2(±82.43),0.14(±0.26),0.66(±0.47),0.61(±0.46),0.39(±1.08),0.67(±0.48),0.69(±0.56),0.66(±0.47)


### Reference stats per assembly algorithm

In [16]:
ref_stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in reference_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    ref_stats_per_assembler_algo=ref_stats_per_assembler_algo.append(row, ignore_index=True)
ref_stats_per_assembler_algo = ref_stats_per_assembler_algo.set_index('Assembler Algorithm')
ref_stats_per_assembler_algo.to_csv("Tables/Results/Reference metrics per assembler algorithm.csv")

In [17]:
ref_stats_per_assembler_algo

,L90,NA50,NG50,Ns,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler Algorithm,,,,,,,,,,,,,,
Single k-mer De Bruijn graph,59.16(±179.99),8841.57(±15783.42),8407.69(±15608.21),0.0(±0.0),1281530.89(±1943438.06),0.35(±0.44),200.97(±381.52),0.03(±0.13),0.47(±0.5),0.46(±0.5),0.03(±0.25),0.47(±0.5),0.47(±0.5),0.47(±0.5)
Multiple k-mer De Bruijn graph,21.51(±55.65),72552.08(±116230.69),72120.23(±116485.25),61.1(±694.43),1639365.75(±2158024.18),0.5(±0.49),57.15(±174.07),0.12(±0.25),0.59(±0.49),0.54(±0.48),1.34(±23.58),0.59(±0.49),0.6(±0.54),0.59(±0.49)


### Reference stats per assembler

In [18]:
ref_stats_per_assembler = pd.DataFrame()
for assembler in reference_pipeline_metrics_df.assembler.unique():
    row={'Assembler': assembler, 
         'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
         'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        std = lala.loc['std',column]
        row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
    ref_stats_per_assembler=ref_stats_per_assembler.append(row, ignore_index=True)
ref_stats_per_assembler = ref_stats_per_assembler.set_index('Assembler')
ref_stats_per_assembler.to_csv("Tables/Results/Reference metrics per assembler.csv")

In [19]:
ref_stats_per_assembler

,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,44.26(±217.35),5388.68(±13665.31),5086.4(±13438.6),0.0(±0.0),Genomic,840447.21(±1648230.31),0.23(±0.38),188.96(±421.28),0.02(±0.12),0.33(±0.47),0.33(±0.47),0.0(±0.0),0.33(±0.47),0.33(±0.47),0.33(±0.47)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,26.58(±60.33),75509.0(±112297.21),70917.21(±103321.28),0.0(±0.0),Metagenomic,1828005.45(±2235991.29),0.56(±0.49),47.59(±88.59),0.14(±0.27),0.62(±0.48),0.61(±0.48),0.17(±0.53),0.63(±0.49),0.63(±0.49),0.62(±0.48)
IDBA-UD,Multiple k-mer De Bruijn graph,43.75(±74.53),52040.9(±91871.34),51122.97(±91883.14),0.0(±0.0),Metagenomic,1821854.84(±2229476.67),0.54(±0.48),74.91(±120.52),0.09(±0.2),0.66(±0.47),0.57(±0.46),0.64(±1.62),0.67(±0.47),0.74(±0.75),0.65(±0.47)
MEGAHIT,Multiple k-mer De Bruijn graph,12.94(±22.35),105537.11(±143662.92),109204.12(±146673.33),0.0(±0.0),Metagenomic,1837433.01(±2247331.45),0.56(±0.49),31.08(±44.97),0.17(±0.3),0.68(±0.46),0.61(±0.45),0.57(±1.17),0.68(±0.47),0.69(±0.47),0.68(±0.47)
metaSPAdes,Multiple k-mer De Bruijn graph,13.56(±20.66),97829.43(±133239.5),96722.68(±132486.45),0.0(±0.0),Metagenomic,1835138.35(±2245485.34),0.56(±0.49),27.24(±38.21),0.16(±0.27),0.68(±0.47),0.65(±0.46),0.17(±0.5),0.69(±0.47),0.69(±0.47),0.68(±0.47)
MINIA,Single k-mer De Bruijn graph,74.05(±131.35),12294.45(±16991.03),11728.97(±16899.85),0.0(±0.0),Genomic,1722614.56(±2112791.43),0.48(±0.47),212.97(±337.68),0.04(±0.13),0.61(±0.49),0.59(±0.49),0.07(±0.35),0.61(±0.49),0.61(±0.49),0.61(±0.49)
SKESA,Multiple k-mer De Bruijn graph,22.76(±44.4),47300.06(±83310.7),46891.18(±83381.95),0.0(±0.0),Genomic,1436160.07(±1959266.05),0.47(±0.47),95.31(±280.36),0.12(±0.26),0.54(±0.5),0.48(±0.48),0.33(±1.49),0.54(±0.5),0.54(±0.5),0.54(±0.5)
SPAdes,Multiple k-mer De Bruijn graph,25.32(±98.82),83133.97(±116730.71),83465.24(±117922.75),0.0(±0.0),Genomic,1808988.64(±2231603.08),0.52(±0.48),72.15(±259.8),0.12(±0.21),0.7(±0.45),0.64(±0.45),8.28(±66.35),0.72(±0.46),0.73(±0.48),0.7(±0.45)
Unicycler,Multiple k-mer De Bruijn graph,11.68(±21.53),104906.65(±137292.36),101808.62(±136709.67),0.0(±0.0),Genomic,1742439.92(±2213254.72),0.54(±0.48),23.51(±35.57),0.16(±0.26),0.56(±0.5),0.54(±0.49),0.12(±0.53),0.56(±0.5),0.56(±0.5),0.55(±0.5)


### Reference stats per reference replicon

In [20]:
reference_pipeline_metrics_df.reference.unique()

array(['Bacillus_subtilis', 'Enterococcus_faecalis',
       'Escherichia_coli_plasmid', 'Escherichia_coli',
       'Lactobacillus_fermentum', 'Listeria_monocytogenes',
       'Pseudomonas_aeruginosa', 'Salmonella_enterica',
       'Staphylococcus_aureus', 'Staphylococcus_aureus_plasmid1',
       'Staphylococcus_aureus_plasmid2', 'Staphylococcus_aureus_plasmid3'],
      dtype=object)

In [22]:
for reference in reference_pipeline_metrics_df.reference.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.reference == reference)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            std = lala.loc['std',column]
            row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(reference))
    print(reference)
    display(ref_stats_per_reference_per_assembler)

Bacillus_subtilis


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,32.5(±50.86),17016.5(±25096.26),16507.83(±25340.74),0.0(±0.0),Genomic,1913231.5(±1791735.3),0.47(±0.44),382.67(±527.71),0.01(±0.01),0.83(±0.38),0.83(±0.38),0.0(±0.0),0.83(±0.38),0.83(±0.38),0.83(±0.38)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,115.78(±163.82),149567.5(±120326.96),147301.83(±118434.41),0.0(±0.0),Metagenomic,3960391.94(±36960.38),0.98(±0.01),165.39(±211.89),0.08(±0.06),1.0(±0.0),0.99(±0.01),0.5(±0.79),1.0(±0.0),1.0(±0.0),1.0(±0.0)
IDBA-UD,Multiple k-mer De Bruijn graph,92.22(±96.95),111081.0(±92895.6),110171.67(±92239.26),0.0(±0.0),Metagenomic,3968675.5(±15308.63),0.98(±0.0),141.56(±137.78),0.05(±0.03),1.0(±0.0),0.98(±0.01),0.83(±1.1),1.0(±0.0),1.0(±0.0),1.0(±0.0)
MEGAHIT,Multiple k-mer De Bruijn graph,38.67(±50.29),205803.83(±100307.84),206013.67(±100176.43),0.0(±0.0),Metagenomic,3983428.61(±6728.47),0.98(±0.0),65.33(±73.39),0.11(±0.06),1.0(±0.01),0.87(±0.21),0.5(±0.79),1.0(±0.0),1.0(±0.0),1.0(±0.0)
metaSPAdes,Multiple k-mer De Bruijn graph,32.83(±21.97),173649.83(±109893.2),170909.5(±107527.12),0.0(±0.0),Metagenomic,3977137.17(±5077.38),0.98(±0.0),60.0(±37.29),0.13(±0.09),1.0(±0.0),0.98(±0.01),0.67(±0.77),1.0(±0.0),1.0(±0.0),1.0(±0.0)
MINIA,Single k-mer De Bruijn graph,193.33(±212.94),40228.33(±21807.72),39618.44(±21736.38),0.0(±0.0),Genomic,3928316.11(±49163.52),0.97(±0.01),278.61(±238.93),0.02(±0.01),1.0(±0.0),1.0(±0.01),0.67(±0.97),1.0(±0.01),1.0(±0.01),1.0(±0.0)
SKESA,Multiple k-mer De Bruijn graph,9.33(±9.92),117416.17(±122527.71),116361.33(±121157.86),0.0(±0.0),Genomic,1984359.17(±2041889.94),0.49(±0.5),16.33(±17.19),0.05(±0.05),0.5(±0.51),0.5(±0.51),0.0(±0.0),0.5(±0.51),0.5(±0.51),0.5(±0.51)
SPAdes,Multiple k-mer De Bruijn graph,27.17(±22.96),214387.33(±100758.97),222639.33(±107608.35),0.0(±0.0),Genomic,3974502.67(±6004.79),0.98(±0.0),49.33(±34.12),0.13(±0.06),1.0(±0.0),0.99(±0.01),0.33(±0.77),1.0(±0.0),1.0(±0.0),1.0(±0.0)
Unicycler,Multiple k-mer De Bruijn graph,28.83(±51.41),180230.33(±139849.68),162359.67(±153864.25),0.0(±0.0),Genomic,2965228.33(±1541741.42),0.73(±0.38),43.5(±61.97),0.11(±0.08),0.83(±0.38),0.83(±0.38),0.0(±0.0),0.83(±0.38),0.83(±0.38),0.83(±0.38)


Enterococcus_faecalis


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,17.61(±40.53),5311.67(±11581.76),5064.17(±11652.13),0.0(±0.0),Genomic,650067.0(±1057244.09),0.23(±0.37),160.39(±299.47),0.01(±0.02),0.33(±0.49),0.33(±0.49),0.0(±0.0),0.33(±0.49),0.33(±0.49),0.33(±0.49)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,9.17(±9.98),83020.17(±87727.63),84170.5(±89247.09),0.0(±0.0),Metagenomic,1401643.72(±1442325.48),0.49(±0.51),19.28(±20.33),0.08(±0.09),0.5(±0.51),0.49(±0.51),0.33(±0.49),0.5(±0.51),0.5(±0.51),0.5(±0.51)
IDBA-UD,Multiple k-mer De Bruijn graph,32.72(±53.94),41221.11(±53482.61),41736.33(±54302.66),0.0(±0.0),Metagenomic,1393139.22(±1433560.52),0.49(±0.5),56.06(±83.83),0.05(±0.06),0.5(±0.51),0.46(±0.48),0.5(±0.79),0.5(±0.52),0.5(±0.52),0.5(±0.51)
MEGAHIT,Multiple k-mer De Bruijn graph,7.0(±7.37),117668.83(±121080.23),118207.17(±121639.12),0.0(±0.0),Metagenomic,1404943.83(±1445728.5),0.49(±0.51),15.83(±16.34),0.11(±0.12),0.5(±0.51),0.48(±0.49),0.33(±0.49),0.5(±0.51),0.5(±0.51),0.5(±0.51)
metaSPAdes,Multiple k-mer De Bruijn graph,7.5(±9.29),103369.67(±111300.82),103369.67(±111300.82),0.0(±0.0),Metagenomic,1396282.67(±1436822.15),0.49(±0.5),15.17(±18.36),0.08(±0.1),0.5(±0.51),0.49(±0.5),0.17(±0.38),0.5(±0.52),0.5(±0.52),0.5(±0.51)
MINIA,Single k-mer De Bruijn graph,59.28(±62.9),14923.44(±13043.93),13404.67(±14286.94),0.0(±0.0),Genomic,1383418.67(±1413662.1),0.49(±0.5),93.5(±95.6),0.02(±0.02),0.83(±0.38),0.83(±0.38),0.0(±0.0),0.83(±0.38),0.83(±0.38),0.83(±0.38)
SKESA,Multiple k-mer De Bruijn graph,15.83(±17.35),49461.0(±51925.56),47018.5(±51361.46),0.0(±0.0),Genomic,1402020.17(±1437301.65),0.49(±0.51),29.83(±29.27),0.06(±0.06),0.83(±0.38),0.69(±0.39),0.33(±0.49),0.83(±0.38),0.84(±0.39),0.83(±0.38)
SPAdes,Multiple k-mer De Bruijn graph,8.0(±9.49),97227.67(±108468.98),98739.33(±109480.51),0.0(±0.0),Genomic,1412023.67(±1443008.6),0.5(±0.51),21.67(±21.27),0.1(±0.1),0.79(±0.38),0.71(±0.43),1.67(±2.7),0.87(±0.41),0.96(±0.52),0.78(±0.38)
Unicycler,Multiple k-mer De Bruijn graph,4.83(±5.12),141763.17(±153616.5),133592.17(±149461.16),0.0(±0.0),Genomic,1396688.5(±1437221.46),0.49(±0.51),11.5(±12.11),0.13(±0.14),0.5(±0.51),0.5(±0.51),0.0(±0.0),0.5(±0.51),0.5(±0.51),0.5(±0.51)


Escherichia_coli_plasmid


...

,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),1056.0(±2429.75),1056.0(±2429.75),0.0(±0.0),Genomic,1056.0(±2429.75),0.17(±0.38),0.17(±0.38),0.17(±0.38),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.38),0.17(±0.38),0.17(±0.38)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),3196.11(±3289.42),3196.11(±3289.42),0.0(±0.0),Metagenomic,3165.83(±3257.62),0.5(±0.51),0.5(±0.51),0.5(±0.52),0.5(±0.51),0.5(±0.51),0.0(±0.0),0.5(±0.52),0.5(±0.52),0.5(±0.51)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),1854.83(±2737.82),1854.83(±2737.82),0.0(±0.0),Metagenomic,1849.83(±2732.15),0.29(±0.43),0.33(±0.49),0.29(±0.43),0.33(±0.48),0.33(±0.48),0.0(±0.0),0.33(±0.49),0.33(±0.49),0.33(±0.48)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),3244.5(±3342.27),3244.5(±3342.27),0.0(±0.0),Metagenomic,3168.17(±3260.02),0.5(±0.51),0.5(±0.51),0.5(±0.52),0.49(±0.51),0.49(±0.51),0.0(±0.0),0.5(±0.52),0.51(±0.53),0.49(±0.51)
metaSPAdes,Multiple k-mer De Bruijn graph,0.17(±0.38),2818.17(±2987.9),2818.17(±2987.9),0.0(±0.0),Metagenomic,3146.0(±3237.52),0.5(±0.51),0.67(±0.77),0.44(±0.47),0.5(±0.51),0.5(±0.51),0.0(±0.0),0.5(±0.52),0.5(±0.52),0.5(±0.51)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),1271.0(±2380.67),1056.0(±2429.75),0.0(±0.0),Genomic,1271.0(±2380.67),0.2(±0.38),0.33(±0.49),0.2(±0.38),0.33(±0.49),0.33(±0.49),0.0(±0.0),0.33(±0.49),0.33(±0.49),0.33(±0.49)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),2287.5(±2787.9),2287.5(±2787.9),0.0(±0.0),Genomic,2819.33(±2928.59),0.44(±0.46),0.83(±1.1),0.36(±0.44),0.5(±0.51),0.49(±0.51),0.0(±0.0),0.5(±0.51),0.5(±0.52),0.5(±0.51)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),2112.0(±3073.41),2112.0(±3073.41),0.0(±0.0),Genomic,2112.0(±3073.41),0.33(±0.48),0.33(±0.49),0.33(±0.48),0.33(±0.49),0.33(±0.49),0.0(±0.0),0.33(±0.49),0.33(±0.49),0.33(±0.49)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),3137.5(±3228.81),3137.5(±3228.81),0.0(±0.0),Genomic,3132.5(±3223.53),0.49(±0.51),0.5(±0.51),0.49(±0.51),0.5(±0.51),0.5(±0.51),0.0(±0.0),0.5(±0.51),0.5(±0.52),0.5(±0.51)


Staphylococcus_aureus_plasmid2


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),398.0(±915.76),398.0(±915.76),0.0(±0.0),Metagenomic,369.33(±849.8),0.17(±0.38),0.17(±0.38),0.18(±0.41),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.18(±0.41),0.18(±0.41),0.17(±0.38)
IDBA-UD,Multiple k-mer De Bruijn graph,0.0(±0.0),385.83(±887.76),385.83(±887.76),0.0(±0.0),Metagenomic,369.33(±849.8),0.17(±0.38),0.17(±0.38),0.17(±0.4),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.4),0.17(±0.4),0.17(±0.38)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),392.83(±903.87),392.83(±903.87),0.0(±0.0),Metagenomic,369.33(±849.8),0.17(±0.38),0.17(±0.38),0.18(±0.41),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.18(±0.41),0.18(±0.41),0.17(±0.38)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),378.5(±870.89),378.5(±870.89),0.0(±0.0),Metagenomic,369.33(±849.8),0.17(±0.38),0.17(±0.38),0.17(±0.39),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.39),0.17(±0.39),0.17(±0.38)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),334.17(±768.88),334.17(±768.88),0.0(±0.0),Genomic,334.17(±768.88),0.15(±0.35),0.17(±0.38),0.15(±0.35),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.38),0.17(±0.38),0.17(±0.38)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),326.83(±752.01),326.83(±752.01),0.0(±0.0),Genomic,326.83(±752.01),0.15(±0.34),0.17(±0.38),0.15(±0.34),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.38),0.17(±0.38),0.17(±0.38)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),369.33(±849.8),369.33(±849.8),0.0(±0.0),Genomic,369.33(±849.8),0.17(±0.38),0.17(±0.38),0.17(±0.38),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.38),0.17(±0.38),0.17(±0.38)


Staphylococcus_aureus_plasmid3


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,0.0(±0.0),526.22(±1210.78),526.22(±1210.78),0.0(±0.0),Metagenomic,498.83(±1147.76),0.17(±0.38),0.17(±0.38),0.18(±0.4),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.18(±0.4),0.18(±0.4),0.17(±0.38)
IDBA-UD,Multiple k-mer De Bruijn graph,0.17(±0.38),281.33(±647.32),281.33(±647.32),0.0(±0.0),Metagenomic,459.5(±1057.26),0.15(±0.35),0.33(±0.77),0.09(±0.22),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.4),0.17(±0.4),0.17(±0.38)
MEGAHIT,Multiple k-mer De Bruijn graph,0.0(±0.0),522.33(±1201.83),522.33(±1201.83),0.0(±0.0),Metagenomic,498.83(±1147.76),0.17(±0.38),0.17(±0.38),0.17(±0.4),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.4),0.17(±0.4),0.17(±0.38)
metaSPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),642.83(±1479.09),642.83(±1479.09),0.0(±0.0),Metagenomic,498.83(±1147.76),0.17(±0.38),0.17(±0.38),0.21(±0.49),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.21(±0.49),0.21(±0.49),0.17(±0.38)
MINIA,Single k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
SKESA,Multiple k-mer De Bruijn graph,0.0(±0.0),463.67(±1066.85),463.67(±1066.85),0.0(±0.0),Genomic,463.67(±1066.85),0.15(±0.36),0.17(±0.38),0.15(±0.36),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.38),0.17(±0.38),0.17(±0.38)
SPAdes,Multiple k-mer De Bruijn graph,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),Genomic,0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0),0.0(±0.0)
Unicycler,Multiple k-mer De Bruijn graph,0.0(±0.0),498.83(±1147.76),498.83(±1147.76),0.0(±0.0),Genomic,498.83(±1147.76),0.17(±0.38),0.17(±0.38),0.17(±0.38),0.17(±0.38),0.17(±0.38),0.0(±0.0),0.17(±0.38),0.17(±0.38),0.17(±0.38)


### Reference stats per distribution

In [23]:
for distribution in reference_pipeline_metrics_df.distribution.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            std = lala.loc['std',column]
            row[column] = "{}(±{})".format(round(mean, 2), round(std, 2))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(distribution))
    print(distribution)
    display(ref_stats_per_reference_per_assembler)

Even


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,74.94(±299.94),7322.22(±14723.55),6909.42(±14616.47),0.0(±0.0),Genomic,1290698.37(±1823393.22),0.37(±0.42),342.06(±543.67),0.04(±0.17),0.5(±0.5),0.5(±0.5),0.0(±0.0),0.5(±0.5),0.5(±0.5),0.5(±0.5)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,29.53(±32.72),109768.25(±118113.5),100966.81(±103048.46),0.0(±0.0),Metagenomic,2524991.72(±2137594.31),0.87(±0.31),57.58(±62.8),0.25(±0.34),0.89(±0.32),0.86(±0.32),0.25(±0.64),0.89(±0.32),0.89(±0.32),0.89(±0.32)
IDBA-UD,Multiple k-mer De Bruijn graph,71.31(±84.58),56068.1(±73615.0),55809.71(±73485.54),0.0(±0.0),Metagenomic,2508186.43(±2123775.36),0.83(±0.34),120.27(±138.59),0.16(±0.27),0.86(±0.35),0.75(±0.38),1.01(±2.14),0.87(±0.35),0.87(±0.35),0.86(±0.35)
MEGAHIT,Multiple k-mer De Bruijn graph,17.47(±17.84),144475.11(±140689.52),151952.72(±144268.39),0.0(±0.0),Metagenomic,2533615.8(±2145228.58),0.88(±0.31),44.06(±44.41),0.3(±0.37),0.89(±0.31),0.79(±0.35),1.08(±1.47),0.89(±0.32),0.9(±0.32),0.89(±0.31)
metaSPAdes,Multiple k-mer De Bruijn graph,19.92(±22.85),134941.08(±125388.71),133991.47(±125026.18),0.0(±0.0),Metagenomic,2529572.39(±2143529.81),0.87(±0.31),38.94(±42.47),0.28(±0.34),0.89(±0.32),0.88(±0.31),0.17(±0.56),0.9(±0.32),0.9(±0.32),0.89(±0.32)
MINIA,Single k-mer De Bruijn graph,82.95(±109.73),18327.94(±18094.39),17669.59(±17958.75),0.0(±0.0),Genomic,2345473.31(±1977463.42),0.72(±0.38),306.45(±390.73),0.08(±0.18),0.81(±0.4),0.79(±0.39),0.0(±0.0),0.81(±0.4),0.81(±0.4),0.81(±0.4)
SKESA,Multiple k-mer De Bruijn graph,38.81(±44.45),84178.14(±88597.63),83571.72(±88877.24),0.0(±0.0),Genomic,2505379.86(±2122746.26),0.85(±0.31),69.67(±78.4),0.23(±0.34),0.89(±0.32),0.77(±0.38),0.33(±0.75),0.89(±0.32),0.89(±0.32),0.88(±0.31)
SPAdes,Multiple k-mer De Bruijn graph,19.72(±20.76),129438.44(±125261.5),129237.0(±124835.36),0.0(±0.0),Genomic,2524523.92(±2139592.08),0.81(±0.37),39.03(±39.76),0.21(±0.26),0.83(±0.37),0.8(±0.38),0.39(±1.01),0.83(±0.38),0.84(±0.38),0.83(±0.37)
Unicycler,Multiple k-mer De Bruijn graph,15.64(±17.26),161424.75(±140701.13),158207.14(±139990.0),0.0(±0.0),Genomic,2521523.86(±2137646.62),0.87(±0.31),32.81(±33.23),0.29(±0.32),0.89(±0.32),0.86(±0.33),0.17(±0.56),0.89(±0.32),0.89(±0.32),0.89(±0.32)


Log


,Algorithm,L90,NA50,NG50,Ns,Type,basepairs,breadth_of_coverage,contigs,contiguity,identity,lowest_identity,misassemblies,multiplicity,parsimony,validity
Assembler,,,,,,,,,,,,,,,,
BCALM2,Single k-mer De Bruijn graph,13.58(±55.31),3455.14(±12284.0),3263.39(±11938.57),0.0(±0.0),Genomic,390196.06(±1313282.67),0.09(±0.27),35.86(±117.16),0.0(±0.01),0.17(±0.37),0.17(±0.37),0.0(±0.0),0.17(±0.37),0.17(±0.37),0.17(±0.37)
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,23.63(±78.89),41249.75(±94968.71),40867.61(±94913.24),0.0(±0.0),Metagenomic,1131019.19(±2120393.96),0.25(±0.43),37.59(±107.82),0.02(±0.06),0.36(±0.48),0.36(±0.48),0.08(±0.36),0.36(±0.48),0.36(±0.48),0.36(±0.48)
IDBA-UD,Multiple k-mer De Bruijn graph,16.19(±49.73),48013.69(±107273.76),46436.22(±107328.74),0.0(±0.0),Metagenomic,1135523.26(±2127500.35),0.25(±0.43),29.56(±76.34),0.02(±0.06),0.46(±0.49),0.4(±0.47),0.28(±0.65),0.47(±0.5),0.61(±0.99),0.44(±0.49)
MEGAHIT,Multiple k-mer De Bruijn graph,8.42(±25.38),66599.11(±136443.17),66455.53(±136838.51),0.0(±0.0),Metagenomic,1141250.22(±2137239.42),0.25(±0.43),18.11(±41.85),0.03(±0.08),0.47(±0.49),0.43(±0.47),0.06(±0.23),0.47(±0.5),0.48(±0.51),0.47(±0.5)
metaSPAdes,Multiple k-mer De Bruijn graph,7.19(±15.94),60717.78(±131029.85),59453.89(±129745.62),0.0(±0.0),Metagenomic,1140704.31(±2136193.13),0.25(±0.43),15.53(±29.22),0.03(±0.07),0.47(±0.5),0.43(±0.48),0.17(±0.44),0.47(±0.5),0.48(±0.51),0.47(±0.5)
MINIA,Single k-mer De Bruijn graph,65.15(±149.9),6260.97(±13386.39),5788.35(±13420.91),0.0(±0.0),Genomic,1099755.82(±2067843.02),0.24(±0.41),119.49(±242.52),0.01(±0.01),0.42(±0.5),0.39(±0.49),0.14(±0.48),0.42(±0.5),0.42(±0.5),0.42(±0.49)
SKESA,Multiple k-mer De Bruijn graph,6.72(±38.29),10421.97(±57890.12),10210.64(±57920.98),0.0(±0.0),Genomic,366940.28(±948388.34),0.1(±0.26),120.94(±387.9),0.01(±0.03),0.19(±0.4),0.19(±0.39),0.33(±1.98),0.19(±0.4),0.19(±0.4),0.19(±0.4)
SPAdes,Multiple k-mer De Bruijn graph,30.92(±138.3),36829.5(±85794.72),37693.47(±90157.5),0.0(±0.0),Genomic,1093453.36(±2096483.13),0.23(±0.41),105.28(±363.08),0.02(±0.05),0.56(±0.48),0.49(±0.47),16.17(±93.38),0.6(±0.51),0.63(±0.55),0.57(±0.48)
Unicycler,Multiple k-mer De Bruijn graph,7.72(±24.54),48388.56(±107839.89),45410.11(±107403.42),0.0(±0.0),Genomic,963355.97(±2011952.12),0.2(±0.39),14.22(±35.55),0.03(±0.06),0.22(±0.42),0.22(±0.41),0.08(±0.5),0.22(±0.42),0.22(±0.42),0.22(±0.42)
